## Darknet Video Test

Using darknet build

In [1]:
import requests
import json
from PIL import Image
import cv2
from io import BytesIO

import time
import io

In [2]:
def darknet(frame):
    url = "http://localhost:44000/score"
    headers = {"Content-Type": "image/jpeg" }

    try:
        r = requests.post(url, headers=headers, data=frame)
        j = json.loads(r.content.decode("utf-8"))
        return j
    except:
        return json.loads("{\"inferences\": []}")

In [4]:
### capture camera
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("rtsp://<ip>")

print('width: {0}, height: {1}'.format(cap.get(3),cap.get(4)))
cap.set(3,416)
cap.set(4,416)

prev_time = 0
FPS = 2
fbuff = None
response = None

while(True):
    ret, frame = cap.read()
    curr_time = time.time() - prev_time
    if (ret) and (curr_time > 1./FPS):
        prev_time = time.time()
        
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)       
        fbuff = cv2.resize(frame, (416, 416), interpolation =cv2.INTER_AREA)
        
        # encode to jpg
        is_success, buffer = cv2.imencode(".jpg", fbuff)
        io_buf = io.BytesIO(buffer)
        pred = darknet(io_buf)

        # highlights
        for p in pred["inferences"]:
            x = int(p["entity"]["box"]["l"] * 416 * 640/416 + 0.5)
            y = int(p["entity"]["box"]["t"] * 416 * 480/416 + 0.5)
            w = int(p["entity"]["box"]["w"] * 416 * 640/416 + 0.5)
            h = int(p["entity"]["box"]["h"] * 416 * 480/416 + 0.5)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 1) 
            cv2.putText(frame, "%s" % p["entity"]["tag"]["value"], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


width: 640.0, height: 480.0
